In [1]:
import re
import numpy as np

### Stworzenie słownika z bazy sjp

In [2]:
with open ("files/polish.stopwords.txt", "r", encoding="utf8") as file:
    lista = file.read()
lista_sp = []
for el in lista.split():
    if len(el) >=3:
        lista_sp.append(el)

In [3]:
D = {}
cores = []
max_length = 10
min_length = 4
with open("files/odm.txt", "r", encoding="utf8") as f:
    while words_list := f.readline():
        words = words_list.replace(", ", ",").replace("\n", "")
        found = False
        for sign in ".:- !?":
            if sign in words:
                found = True
        words_list = words.split(",")
        core = words_list[0].lower()
        if found or len(core) > max_length or len(core) < min_length or core in lista_sp:
            continue 
        cores.append(core)
        for word in words_list:
            word = word.lower()
            if word not in D:
                D[word] = core         
cores = list(set(cores))

In [4]:
len(cores)

138163

In [5]:
# lista_sp

### Funkcja do lematyzacji
(kto by się spodziewał)

In [6]:
def clean(tekst):
    res = ""
    tekst = tekst.replace('\n', ' ')
    tekst = tekst.lower()
    for el in tekst:
        if el not in [".",",","?","!",";",":","%", "&", ")",">","]","$","@","(", "[", "<", "0","1","2","3","4","5","6","7","8","9"]:
            res += el 
    return res

def lemmatisation(text, D, stopwords):
#     text = re.sub("[,.:]", "", text)
#     text = text.replace("\n", " ")

    text = clean(text)
#     print(text)
#     return " ".join([D[word] if word in D else word for word in text.split(" ") ])
    return " ".join([D[word] for word in text.split(" ") if word in D and word not in stopwords])    

In [7]:
text = """Po pierwsze przekazać swój protest w związku z naruszeniem polskiej przestrzeni powietrznej oraz dodatkowo w związku z zachowaniem ambasadora drogą pisemną, czyli za pomocą noty dyplomatycznej, którą możemy przesłać zarówno Ambasadzie Rosji w Warszawie, jak i dla zwiększenia efektu poprzez naszą ambasadę rosyjskiemu MSZ.
Po drugie można wydać dyspozycję ambasadorowi RP w Moskwie, by ten równie ostentacyjnie, co jego rosyjski odpowiednik w Warszawie, zignorował jedno z najbliższych zaproszeń do rosyjskiego MSZ, a następnie udzielił wywiadu wybranemu polskiemu medium, w którym wyjaśni, że nie odpowiadała mu forma zaproszenia.
Trzecia opcja, na którą wydawał się niestety wskazywać rzecznik MSZ Paweł Wroński, to wydalenie ambasadora Rosji. Problem polega na tym, że to, na co wskazywał rzecznik MSZ, opłaca się Rosji, ale nie Polsce. Jeśli bowiem Polska wydali ambasadora Rosji, to Rosjanie w ramach wzajemności, czyli zasady dyplomacji, którą bezwzględnie stosują, wydalą z Rosji ambasadora RP w Moskwie Krzysztofa Krajewskiego. Byłoby to głęboko niezgodne z żywotnymi interesami narodowymi RP.
Utrzymywanie kanału komunikacji z Moskwą na poziomie ambasadora jest w sytuacji toczącej się za naszą granicą wojny bardziej, a nie mniej istotne, niż w czasach pokoju.
Co więcej, obecność ambasadora pozwala wnikliwej monitorować to, co dzieje się pomiędzy np. Waszyngtonem a Moskwą. Mówiąc wprost, jeśli założyć, że Amerykanie nie zawsze lub nie od razu nam wszystko mówią, to warto mieć nasłuch nie tylko w Departamencie Stanu USA czy też w Białym Domu, ale również Ambasadzie USA w Moskwie.
W dodatku obecny ambasador w Moskwie należy do wąskiego grona ambasadorów państw NATO w Moskwie, których głos dociera nie tylko do ich stolic, ale również do Kwatery Głównej paktu."""
new_text = lemmatisation(text, D, lista_sp)
new_text

'pierwsza przekazać protest związek naruszyć polski przestrzeń powietrzny dodatkowy związek zachować ambasador droga pisemny pomoc nota przesłać zarówno ambasada rosja warszawa zwiększyć efekt poprzez ambasada rosyjski druga wydać dyspozycja ambasador moskwa równia rosyjski warszawa zignorować najbliższy zaprosić rosyjski następny udzielić wywiad wybrać polski medium wyjaśnić odpowiadać forma zaprosić trzeci opcja wydawać niestety wskazywać rzecznik paweł wroński wydalić ambasador rosja problem polegać wskazywać rzecznik opłacać rosja polska polska wydać ambasador rosja rosjanin rama wzajemność zasada dyplomacja stosować wydalić rosja ambasador moskwa krzysztof krajewski głęboki zgodny żywotny interes narodowy utrzymywać kanał moskwa poziom ambasador sytuacja toczyć granica wojna mniej istotny czas pokój obecność ambasador pozwalać wnikliwy dziać pomiędzy waszyngton moskwa mówić wprost założyć amerykan mówić warta miecia nasłuch stan biali ambasada moskwa dodatek obecny ambasador moskw

### Zliczanie powtórzeń

In [8]:
occurences = {}
for word in new_text.split(" "):
    if word not in occurences:
        occurences[word] = 1
    else:
        occurences[word] += 1

In [9]:
# {word:occurences[word] for word in occurences.keys() if occurences[word] > 1}
sorted(((occurence, word) for word, occurence in occurences.items()), reverse=True)[:4]

[(9, 'ambasador'), (7, 'moskwa'), (5, 'rosja'), (3, 'rosyjski')]

#### todo
- pod prezentację stworzyć histogram
- wyrzucić spójniki
- rozdzielic na slownik z rzeczownikami i z czasownikami


In [10]:
D["egzekucja"]

'egzekucja'

Dzisiaj przeprowadzono egzekucję na krześle elektrycznym

### Bag-of-words

In [11]:
len(cores)

138163

In [12]:
def vectorize(sentence_L, all_words):
    vector = np.zeros(len(all_words))
    for word in sentence_L:
        if word in all_words:
            idx = all_words.index(word)
            vector[idx] += 1
    return vector

In [13]:
text_L = new_text.split(" ")

In [14]:
X = vectorize(text_L, cores)

In [15]:
X

array([0., 0., 0., ..., 0., 0., 0.])

###### Zdanie
Marcinowi i o rok młodszemu Krzyśkowi ciężko bronić się podczas procesu. Konsekwentnie mówią, że z zabójstwem braci Nefel nie mają nic wspólnego. Nie było ich na miejscu zbrodni. Gdy doszło do morderstwa, byli ze swoimi rodzinami. Ich zeznania są spójne. Nie zmieniają się.

Kontekst `Zbrodnia`

In [24]:
with open('files/zdania.txt', 'r', encoding='utf8') as f:
    sentences = []
    sentence = ""
    count_breaks = 0
    while line := f.readline():
        if line == '\n':
            count_breaks += 1
        else:
            sentence += line
        if count_breaks == 2:
            sentences.append(sentence)
            sentence = ""
            count_breaks = 0
        

In [36]:
contexts = []
with open('files/konteksty.txt', 'r', encoding='utf8') as f:
    while word := f.readline():
        word = word.replace("\n", "")
        contexts.append(word)
contexts   

['markiza',
 'whiskey',
 'trasa',
 'szczepienia',
 'wypadek',
 'ropa',
 'wypadek',
 'sondaż',
 'wsparcie',
 'banan',
 'zwierzęta',
 'zwierzęta',
 'zwierzęta',
 'sport',
 'sport',
 'astronomia',
 'astronomia',
 'astronomia',
 'historia',
 'historia']

In [44]:
contexts_set = list(set(contexts))
contexts_set

['astronomia',
 'trasa',
 'zwierzęta',
 'sport',
 'wypadek',
 'szczepienia',
 'sondaż',
 'banan',
 'historia',
 'whiskey',
 'ropa',
 'markiza',
 'wsparcie']

In [45]:
def vectorize_output(word, contexts_set):
    out = np.zeros(len(contexts_set))
    index = contexts_set.index(word)
    out[index] = 1
    return out

In [46]:
vectorize_output("historia", contexts_set)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [25]:
len(sentences)

20

In [26]:
sentences[2]

'Ruszając rowerem z serca Szwajcarii Holsztyńskiej, po 45 km dojeżdżamy do Travemünde, nadmorskiej dzielnicy Lubeki z dziką przyrodą oraz niezapomnianym widokiem na Zatokę Lubecką, który rozciąga się ze stromego brzegu Brodten. Klify w tym miejscu osiągają nawet 20 metrów wysokości. Czterokilometrowa ścieżka wzdłuż wybrzeża wiedzie od Travemünde do Niendorf, górą, gdzie spokojnie można przejechać rowerem albo wózkiem, lub dołem – przez dziką i kamienistą plażę. Jedyne schody łączące obie ścieżki znajdują się około kilometra przed Niendorfem.\nTuż obok Travemünde, u ujścia rzeki Trave, położony jest półwysep Priwall z dziewiczą plażą otoczoną wydmami oraz rzadkimi gatunkami ptaków, które można obserwować, przemierzając turystyczne szlaki. Wzdłuż wybrzeża prowadzi trasa rowerowa, a liczne punkty dostępu do plaży zachęcają do zatrzymania się i kąpieli. Obszar ochrony przyrody "Südlicher Priwall" z przyległymi strefami płytkich wód Pötenitzer Wiek to miejsce występowania rzadkich gatunków 

In [19]:
lemma = lemmatisation(sentences[2], D, lista_sp)
print(lemma)
vec = vectorize(lemma, cores)
# vec

ruszać rower serce szwajcaria dojeżdżać nadmorski dzielnica lubeka dzika przyroda zapomnieć widok zatoka lubecki rozciągać stromy brzeg klif miejsce osiągać metr wysokość ścieżek wzdłuż wybrzeże wieść góra spokojny przejechać rower wózek dołem dzika kamienista plaża jedyne schody łączyć obie ścieżek znajdować kilometr ujście rzeka położyć półwysep dziewiczy plaża otoczyć wydma rzadki gatunek ptak obserwować szlak wzdłuż wybrzeże prowadzić trasa rowerowy liczny punkt dostęp plaża zachęcać zatrzymać kąpiel obszar ochrona przyroda przyległy strefa płytki woda wiek miejsce występować rzadki gatunek roślina ptak warta skorzystać oferta warsztat odwiedzić akwarium morze bałtycki dowiedzieć wszystek morski stworzenie podwodne świat bałtyk nakarmić żarłoczny płastuga


In [47]:
vec.shape

(138163,)

In [48]:
words = []
for sentence in sentences:
    words += lemmatisation(sentence, D, lista_sp).split(" ")
words = sorted(list(set(words)))

In [49]:
len(words)

792

In [51]:
vec = vectorize(lemmatisation(sentences[2], D, lista_sp).split(" "), words)

In [22]:
sum(vectorize(lemmatisation(sentences[2], D, lista_sp).split(" "), words))

99.0

In [54]:
X = []
Y = []
for sentence, context in zip(sentences, contexts):
    x = vectorize(lemmatisation(sentence, D, lista_sp).split(" "), words)
    y = vectorize_output(context, contexts_set)
    X.append(x)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

In [63]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
Y

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 

### Sieć neuronowa

In [66]:
from sklearn.neural_network import MLPClassifier

C:\Programs\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [97]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 5, 5, 5), random_state=1, max_iter=10)
clf.fit(X, Y)

C:\Programs\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 5, 5, 5), max_iter=10,
              random_state=1, solver='lbfgs')

In [96]:
for i in range(len(contexts)):
    print("pred:{}, oryg:{}".format(contexts_set[clf.predict_proba([X[i]])[0].argmax()], contexts[i]))

pred:sport, oryg:markiza
pred:sport, oryg:whiskey
pred:historia, oryg:trasa
pred:zwierzęta, oryg:szczepienia
pred:wypadek, oryg:wypadek
pred:sport, oryg:ropa
pred:wypadek, oryg:wypadek
pred:sondaż, oryg:sondaż
pred:sport, oryg:wsparcie
pred:sport, oryg:banan
pred:zwierzęta, oryg:zwierzęta
pred:zwierzęta, oryg:zwierzęta
pred:zwierzęta, oryg:zwierzęta
pred:sport, oryg:sport
pred:sport, oryg:sport
pred:astronomia, oryg:astronomia
pred:astronomia, oryg:astronomia
pred:astronomia, oryg:astronomia
pred:historia, oryg:historia
pred:historia, oryg:historia


In [94]:
sentences[17]

'Supernowa – termin określający kilka rodzajów kosmicznych eksplozji powodujących powstanie na niebie niezwykle jasnego obiektu, który już po kilku tygodniach lub miesiącach staje się niemal niewidoczny. \nSą dwie możliwe drogi prowadzące do takiego wybuchu: w jądrze masywnej gwiazdy przestały zachodzić reakcje termojądrowe bądź zachodzi proces pochłaniający promieniowanie (kreacja par, fotodezintegracja), a zmniejszenie ciśnienia promieniowania powoduje zapadanie się gwiazdy pod własnym ciężarem, bądź też biały karzeł tak długo pobierał materię z sąsiedniej gwiazdy, aż przekroczył masę Chandrasekhara, co doprowadziło do  eksplozji termojądrowej.\nW obydwu przypadkach eksplozja supernowej z ogromną siłą wyrzuca w przestrzeń większość lub całą materię gwiazdy.\n'